#### 토픽 모델링
* https://happy-obok.tistory.com/5
* 토픽 모델링(Topic Modeling)이란 기계 학습 및 자연어 처리 분야에서 토픽이라는 문서 집합의 추상적인 주제를 발견하기 위한 통계적 모델 중 하나로, 텍스트 본문의 숨겨진 의미 구조를 발견하기 위해 사용되는 텍스트 마이닝 기법입니다.
* 청원 데이터에서 각 카테고리별로 숨겨진 토픽들을 찾아내기 위한 방법

#### 토픽 모델링 방법론: ** 잠재 디리클레 **

* 토픽 모델링은 문서 집합에서 토픽을 발견하기 위해 사용되는 통계적 모델링 방법이며, 이 방법은 <u>문서들이 하나 이상의 토픽으로 구성</u>되어 있으며, <u>각 토픽은 특정 단어들의 분포로 표현</u>될 수 있다는 가정 하에 작동함.
    * <u>잠재 디리클레 할당(Latent Dirichlet Allocation, LDA)은 토픽 모델링에서 가장 널리 사용되는 알고리즘</u> 중 하나임.

#### *토픽 모델링링 과정*
* **전처리**: 문서 집합에 대해 텍스트 정제(불용어 제거, 어간 추출 등), 토큰화 등의 전처리 작업을 수행함.
* **모델 선택 및 설정**: LDA와 같은 토픽 모델을 선택하고, 모델의 파라미터(예: 토픽의 수)를 설정함.
* **모델 학습**: 전처리된 데이터에 대해 토픽 모델을 학습시키며, 이 과정에서 각 문서에 대한 토픽 분포와 각 토픽에 대한 단어 분포가 추정됨.
* **결과 해석 및 평가**: 학습된 모델을 기반으로 추출된 토픽과 그 구성 요소인 단어들을 해석하고 모델의 성능을 평가함.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
all_df_2018 =  pd.read_csv("/content/drive/MyDrive/download/all_df.2018")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df_copy_18 = all_df_2018[all_df_2018['num_agree']>500] # 동의수가 500 이상인, 청원이 어느정도 진행된 데이터만을 사용

#### 한글 전처리
* 어미,조사,구두점 제거 -> 어간 추출
* 이 과정이 상당히 오래걸림
* 다음엔 병렬적 처리 방식으로 함수를 설정해보고 싶음

In [ ]:
# 형태소 분석기에서 Okt 불러오기
from konlpy.tag import Okt
okt = Okt()

# 조사,어미,구두점 제거, 어간 추출
def okt_clean(text):
    '''
    텍스트가 들어오면 텍스트에 있는 단어들에게 명사,조사 등의
    이름을 지정하여, 해당 조사,어미,구두점을 제거하는 함수

    text : str

    return : str
    '''
    clean_txt = []
    for word in okt.pos(text,stem=True):
        if word[1] not in ['Josa','Eomi','Punctuation']:
            clean_txt.append(word[0])

    return ' '.join(clean_txt)

from tqdm import tqdm
tqdm.pandas


df_copy_18['doc'] = df_copy_18['doc'].apply(okt_clean)

#### 불용어 처리
*   불용어 처리는 토픽 모델링을 반복해서 돌려가면서, 의미가 있는 단어와 그렇지 않는 단어를 구별하는 작업을 반복

In [ ]:
def remove_stopwords(text):
    '''
    텍스트를 공백을 구분하고, 설정한 불용어를 제거하는 함수

    text : str

    return : str
    '''
    tokens = text.split(' ')
    stops = ['합니다','하다','있다','이다','되다','하지만','그리고','그런데','저는','제가','정말','너무','합니다','않다','없다','되어다','받다','아니다']
    meaningful_words = [word for word in tokens if not word in stops]
    return ' '.join(meaningful_words)

df_copy_18['doc'] = df_copy_18['doc'].apply(remove_stopwords)

#### LDA 모델에 학습시키기 위해, 텍스트들을 벡터화
*   텍스트 데이터를 숫자형 데이터로 변환하는 과정
*   '나는 병원을 갔습니다' -> 0 2 0 1

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer() # 단어들의 출현 빈도로 여러 문서를 벡터화하기 위해 cv를 불러온다
doc_cv = cv.fit_transform(df_copy_18['doc'])
cv_cols = cv.get_feature_names_out()
pd.DataFrame(doc_cv.toarray(),columns=cv_cols).sum().sort_values()

#### 잠재 디리클레 기법 적용

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation # 잠재 디리클레 기법 적용

NUM_TOPICS = 15 # 카테고리는 15개로 설정
LDA_model = LatentDirichletAllocation(n_components=NUM_TOPICS,random_state=42)

LDA_model.fit(doc_cv)

#### 잠재 디리클레 기법이 적용된 topic들을 시각화: 15개의 주제들을 지정




In [ ]:
!pip install scikit-learn==1.0.0  # Adjust version as needed
!pip install pyLDAvis==3.4.1      # 코랩 내에 pyLDAvis 와 사이킷런 버전 호환 문제 해결

In [ ]:
import pyLDAvis

pyLDAvis.enable_notebook()
prepared_data = pyLDAvis.prepare(
    topic_term_dists=LDA_model.components_ / LDA_model.components_.sum(axis=1)[:, np.newaxis],  # 주제-단어 분포
    doc_topic_dists=LDA_model.transform(doc_cv),  # 문서-주제 분포
    doc_lengths=doc_cv.sum(axis=1).A1,  # 각 문서의 총 단어 수
    vocab=cv.get_feature_names_out(),  # 어휘 목록
    term_frequency=np.asarray(doc_cv.sum(axis=0)).ravel(),  # 전체 데이터셋에서 각 단어의 빈도
    mds='tsne'  # 차원 축소 기법 선택
)
pyLDAvis.display(prepared_data)


이 시각화는 특정 주제(Topic 15)에 대한 상위 30개의 관련 용어를 보여줍니다. 다음은 그래프에 대한 상세한 설명입니다.

주요 구성 요소:
제목:

"Top-30 Most Relevant Terms for Topic 15 (2.4% of tokens)"는 이 차트가 Topic 15와 관련된 상위 30개 용어를 보여주며, 이 주제가 전체 데이터셋에서 2.4%를 차지한다는 것을 의미합니다.
관련성 메트릭 슬라이더 (λ):

상단의 슬라이더는 λ 값을 조정하여 관련성 메트릭을 설정합니다. 이 값은 0에서 1까지 조정 가능합니다. 현재 λ 값은 1로 설정되어 있습니다. 관련성 메트릭은 주제 내 용어 빈도와 전체 코퍼스에서의 용어 빈도의 균형을 결정합니다.
λ = 1일 때, 관련성은 주제 내 용어의 빈도에만 기반합니다.

λ = 0일 때, 해당 용어가 전체 코퍼스에서 나타나는 빈도 대비 특정 주제에서 얼마나 자주 나타나는지를 의미합니다. 쉽게 말해, 특정 주제에 대해 얼마나 독특한 용어인지를 보여줍니다.

빨간색 막대: 선택된 주제(Topic 15) 내에서의 예상 용어 빈도를 나타냅니다.
파란색 막대: 전체 코퍼스에서의 전체 용어 빈도를 나타냅니다.
용어는 y축에 나열되어 있으며, 해당 용어의 빈도는 x축에 표시됩니다.
용어와 빈도의 설명:
난민, 불법, 한국, 택배, 제주도 등과 같은 용어들이 나열되어 있습니다. 각 용어의 빨간색 막대는 Topic 15에서 얼마나 자주 나타나는지를 나타내고, 파란색 막대는 데이터셋 전체에서의 빈도를 나타냅니다.

예시 분석:

아이: 이 용어는 파란색 막대가 크고, 빨간색 막대도 상당히 크기 때문에 전체 코퍼스에서 높은 빈도를 가지고 있으며 Topic 15에서도 높은 관련성을 가집니다.
예멘: 이 용어는 파란색 막대가 작고 빨간색 막대가 크기 때문에, 전체 코퍼스에서는 자주 나타나지 않지만 Topic 15에서는 특히 관련성이 높습니다.
공식:

이 공식은 주제 내 용어의 확률과 그 용어의 독특성 간의 균형을 맞춥니다.
해석:
빨간색 막대가 파란색 막대에 비해 높은 용어는 Topic 15에 더 특화된 용어입니다.
파란색 막대가 큰 용어는 전체 코퍼스에서 일반적으로 많이 나타나는 용어지만, 빨간색 막대도 크다면 Topic 15에서도 중요한 용어입니다.
이 시각화는 주어진 코퍼스에서 Topic 15를 정의하는 주요 테마와 특정 용어를 이해하는 데 도움을 줍니다.







#### λ의 의미
*   λ = 1 일수록 토픽 별로 가장 자주 등장하는 단어들을 우선적으로 키워드로 선택한다는 의미 이고
*   λ = 0 일수록 토픽 간에 차이가 많이 나는 단어를 선택한다는 의미(해당 토픽에서 많이 등장한 단어)